In [1]:
import sys
import os

openagent_dir = os.path.dirname(os.path.abspath(""))
sys.path.append(openagent_dir)

In [2]:
from openagent.agent.completion import TextCompletionAgent
from openagent.agent.chat import ChatAgent

from openagent.llms._openai import OpenAI
from openagent.vectorstores.chroma import Chroma
from openagent.text_splitter import CharacterTextSplitter
from openagent.knowledgebase.document_loaders.file.base import SimpleDirectoryReader
from openagent.knowledgebase.doc_loader import document_loader

from openagent.knowledgebase.base import SimpleKnowledgeBase
from openagent.vectorstores.embeddings.openai import OpenAIEmbeddings

# load .env file in format :
# OPENAI_API_KEY=sk-xxx
from dotenv import load_dotenv
load_dotenv()

llmcc = OpenAI(
    model="gpt-3.5-turbo"
)
llmtc = OpenAI(
    model="text-davinci-003"
)

SimpleDirectoryReader = document_loader('file_directory')

reader = SimpleDirectoryReader('test_data/meteoric')
raw_docs = reader.load_data()

splitter = CharacterTextSplitter(chunk_overlap=40, chunk_size=1024)

openai_embeddings = OpenAIEmbeddings()
chroma = Chroma(embedding_function=openai_embeddings)

knowledge = SimpleKnowledgeBase(raw_data=raw_docs, data_transformer=splitter, vector_store=chroma)

c:\Users\DELL\anaconda3\envs\openagent\Lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.22) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [3]:
# raw_docs

In [4]:
reader = SimpleDirectoryReader('test_data/paul_graham')
raw_docs = reader.load_data()

knowledge.add_data(raw_docs)

2023-08-31 19:21:23,249 - 18148 - text_splitter.py-text_splitter:138 - WARNING: Created a chunk of size 1203, which is longer than the specified 1024
2023-08-31 19:21:23,250 - 18148 - text_splitter.py-text_splitter:138 - WARNING: Created a chunk of size 1025, which is longer than the specified 1024


In [5]:
knowledge.references

[{'source': 'test_data\\meteoric', 'loader_key': 'file_directory'},
 {'source': 'test_data\\paul_graham', 'loader_key': 'file_directory'}]

In [6]:
# formatted output with text completion model without knowledge base

text_template = '''
City: "{{city}}"
Average temperature (C): "{{gen 'avg_temp'}}"
Population: "{{population}}"
Famous for (in 10 words): {{gen "famous_for" temperature=0.7 max_tokens=30}}'''

agent = TextCompletionAgent(
    llm=llmtc,
    prompt_template = text_template,
    knowledgebase = knowledge,
    return_complete=True,
    input_variables={'extras': ['city', 'population'], 'knowledge_variable': 'city'}
    )

out = agent.run(city = 'New Delhi', population = "2 crore")

print(out)


City: "New Delhi"
Average temperature (C): "25.5""
Population: "2 crore"
Famous for (in 10 words):  "Historical monuments, culture, food"


In [7]:
agent.export_agent_config('exported_config.agent', export_json = True)

In [8]:
new_agent = TextCompletionAgent.load_from_config('exported_config.agent')

new_out = new_agent.run(city = 'New Delhi', population = "2 crore")

2023-08-31 19:21:28,362 - 18148 - text_splitter.py-text_splitter:138 - WARNING: Created a chunk of size 1203, which is longer than the specified 1024
2023-08-31 19:21:28,364 - 18148 - text_splitter.py-text_splitter:138 - WARNING: Created a chunk of size 1025, which is longer than the specified 1024


In [9]:
new_out

'\n"Monuments, culture, history, food, shopping, politics, education, festivals, art, diversity"'